<a href="https://colab.research.google.com/github/fastforwardlabs/whisper-openai/blob/master/WhisperDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://blog.fastforwardlabs.com/images/cloudera-fast-forward-logo.png" width=300>

# Make your own recordings and transcriptions with OpenAI's Whisper!
_a fun diversion brought to you by [Melanie](https://www.linkedin.com/in/melanierbeck/), ML Research Manager at Cloudera Fast Forward Labs_


[OpenAI](https://openai.com/) [recently released](https://openai.com/blog/whisper/) Whisper, an automatic speech recognition (ASR) system that was trained on a colossal heap of audio data collected from the web. 

## What makes Whisper unique? 
Speech-to-text technology isn't new but Whisper might usher in the  next-generation of ASR systems in terms of the quality and capabilities delivered by a single model (rather than a collection of models, as most ASR systems are today.)  So what makes Whisper special? 

1. It's "weakly supervised," meaning that it was trained on (audio, transcript) pairs wherein the transcriptions were _not_ human-validated, a typical hallmark of gold-stardard supervised audio datasets.  
2. This allows Whisper to be trained on a MASSIVE amount of data scraped from the web (680K hours) -- orders of magnitude larger than most audio datasets (5-30K hours)
3. The data contains audio snippets in dozens of languages allowing Whisper to be natively multilingual. 
4. Whisper is also multitask -- it can perform transcription, translation, voice detection, and language detection. 

You can learn more about Whisper in OpenAI's [recent post]((https://openai.com/blog/whisper/)), their [paper](https://cdn.openai.com/papers/whisper.pdf), and their open-source [codebase](https://github.com/openai/whisper). In fact, this notebook made use of OpenAI's [LibriSpeech](https://colab.research.google.com/github/openai/whisper/blob/master/notebooks/LibriSpeech.ipynb) Colab example as a starting point, so you should check that out as well!


This notebook allows you to play with Whisper by first creating your own audio recording, processing the audio into a format the model will understand, and feeding it into Whisper for translation or transcription! Let's get started. 




## Installs and imports 
The commands below will install the Python packages needed to record audio snippets and use Whisper models for speech-to-text transcription.

In [1]:
! pip install git+https://github.com/openai/whisper.git
! pip install sounddevice wavio
! pip install ipywebrtc notebook

  Cloning https://github.com/openai/whisper.git to c:\users\harec\appdata\local\temp\pip-req-build-du6xyzhk
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numba-0.59.1-cp311-cp311-win_amd64.whl (2.6 MB)
  Using cached torch-2.2.1-cp311-cp311-win_amd64.whl (198.6 MB)
  Using cached more_itertools-10.2.0-py3-none-any.whl (57 kB)
  Using cached tiktoken-0.6.0-cp311-cp311-win_amd64.whl (798 kB)
  Using cached llvmlite-0.42.0-cp311-cp311-win_amd64.whl (28.1 MB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached Jinja2-3.1.3-py3-n

  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\harec\AppData\Local\Temp\pip-req-build-du6xyzhk'

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sounddevice-0.4.6-py3-none-win_amd64.whl (199 kB)
  Using cached wavio-0.0.8-py3-none-any.whl (9.4 kB)
  Using cached cffi-1.16.0-cp311-cp311-win_amd64.whl (181 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached ipywebrtc-0.6.0-py2.py3-none-any.whl (260 kB)
  Using cached notebook-7.1.2-py3-none-any.whl (5.0 MB)
     -------------------------------------- 383.2/383.2 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 59.0/59.0 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 11.4/11.4 MB 5.3 MB/s eta 0:00:00
  Using cached anyio-4.3.0-py3-none-any.whl (85 kB)
     -------------------------------------- 257.4/257.4 kB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 78.4/78.4 kB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 54.5/54.5 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 5.9 MB/s eta 0:00:00
     ---------------------------------------- 58.5/58.5 kB 3.0 MB/s eta 0:00:00
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
     ---------------------------------------- 69.1/69.1 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 11.0/11.0 MB 5


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


We also need the following in order to record audio from this notebook and process the resulting files. 

In [2]:
!apt install ffmpeg
!apt-get install libportaudio2

'apt' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

ModuleNotFoundError: No module named 'pandas'

## Make your recording

First, we need to enable some Colab widgets so that we can make an audio recording. 

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

### Time to record! 

Press the circle button and start speaking. It may not look it, but the widget will be capturing sound. Click the circle button again when you are finished. The widget will immediately begin to play back what it captured. 

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

The audio format captured above is not readable by PyTorch. In this step, we convert our recording into a format that PyTorch can understand. 

In [ ]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav my_recording.wav -y -hide_banner -loglevel panic

### Alternatively... 
If you don't want to make your own recording, you can instead upload an audio file to this notebook. 


## Select options

Whisper is capable of performing transcriptions for many languages (though it performs better for some languages and worse for others.) 

Whisper is also capable of detecting the input language. However, to be on the safe side, we can explicitly tell Whisper which language to expect. 

In [ ]:
language_options = whisper.tokenizer.TO_LANGUAGE_CODE 
language_list = list(language_options.keys())

In [ ]:
lang_dropdown = widgets.Dropdown(options=language_list, value='english')
output = widgets.Output()
display(lang_dropdown)

Dropdown(options=('english', 'chinese', 'german', 'spanish', 'russian', 'korean', 'french', 'japanese', 'portu…

Whisper is also capable of several tasks, including English-only transcription, Any-to-English translation, and non-English transcription. 

Below you can select either "transcription" (which will yield text in the same language as the input language) or "translation" (which will transcribe from non-English to English). 

![Whisper capabilities](https://cdn.openai.com/whisper/draft-20220920a/asr-training-data-desktop.svg)

Image from [Introducing Whisper](https://openai.com/blog/whisper/) by OpenAI

In [ ]:
task_dropdown = widgets.Dropdown(options=['transcribe', 'translate'], value='transcribe')
output = widgets.Output()
display(task_dropdown)

Dropdown(options=('transcribe', 'translate'), value='transcribe')

## Load Whisper model

Whisper comes in five model sizes, four of which also have an optimized English-only version. This notebook loads "base"-sized models (bigger than "tiny" but smaller than the others), which require about 1GB of RAM. 

If you selected English above, the cell below will load the optimized English-only version. Otherwise, it will load the multilingual model. 


In [ ]:
if lang_dropdown.value == "english":
  model = whisper.load_model("base.en")
else:
  model = whisper.load_model("base")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|███████████████████████████████████████| 139M/139M [00:09<00:00, 14.9MiB/s]


Model is English-only and has 71,825,408 parameters.


Finally, let's set the rest of our task and language options below and see what we've got. Check that your task and language settings are correct, but don't worry about the other defaults. 

In [ ]:
options = whisper.DecodingOptions(language=lang_dropdown.value, task=task_dropdown.value, without_timestamps=True)
options

DecodingOptions(task='transcribe', language='english', temperature=0.0, sample_len=None, best_of=None, beam_size=None, patience=None, length_penalty=None, prompt=None, prefix=None, suppress_blank=True, suppress_tokens='-1', without_timestamps=True, max_initial_timestamp=0.0, fp16=True)

## Take Whisper for a test drive

All that's left to do now is feed our audio into Whisper. 

The cell below performs the last processing steps to make this happen. First, it loads our PyTorch-ready audio file. Then it pads the audio into 30 sec segments. It creates a log-mel spectrogram of the audio and this is fed into Whisper along with the options we set above. 



_Note: if you chose to upload your own audio file rather than create one through this notebook, you'll need to update the audio filename below._

In [ ]:
audio = whisper.load_audio("my_recording.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)
result = model.decode(mel, options)

In [ ]:
result.text

'The tree was in Virginia. It was a long way away from anywhere on the back of an old farm. To get to the farm they had to drive for almost an hour south from Blacksburg, to drive roads with names like Pennywinkle Branch and Rooster Spur. They got turned around twice and Mr. Nancy and Chernobyl both lost their tempers with shadow and with each other.'

### How well did Whisper do??

I read aloud the snippet above from one of my favorite books. For the record, Whisper is pretty dang close, making only two small mistakes -- both on proper names.   